In [ ]:
import numpy as np	
from itertools import combinations	
from itertools import product	
	
from stats import Stats	
from character import Character	
	
# Creating an object of the class	
character = Stats(0, 'character base Stats', (15, 20, 0, 150))	
	
dmgs = np.arange(0,15,1)	
crits = np.arange(0,160,10)	
maxis = np.arange(0,140,10)	
crit_dmgs = np.arange(0,50,5)	
	
	
combs = list(product(dmgs, crits, maxis, crit_dmgs))	
combs_len = len(combs)	
	
#	 Generating items
print(f'Generating {combs_len} combinations')	
	
items = []	
for index, group in enumerate(combs):	
		items.append(Stats(index, f'random item {index}', group))
print('Done')	
	
#	 Generating items conbinations
print(f'Generating {combs_len} combinations')	
	
results = []	
for item in items:	
		results.append(character.equip(item))
print('Done')	
	
ordered_res = sorted(results)	
	
print(Stats.headers())	
print(ordered_res[0])	
print(ordered_res[-1])

In [1]:
%reset -f

import numpy as np
from itertools import combinations
from itertools import product
from itertools import combinations_with_replacement

from stats import Stats
from character import Character

# File path and name
file_path = "C:\\Users\\Nicola Basso\\Documents\\els_stat_testing\\res.txt"

# Creating an object of the class

top_rigo_e9r15      = {'phy_atk':1155,	'mag_atk':0,	'phy_def':1883,	'mag_def':1734,	'hp':79741,	'specific':5+5,	'adapt':3,	'crit_dmg':3+3	}
bottom_rigo_e9r15   = {'phy_atk':0,	    'mag_atk':1662,	'phy_def':1708,	'mag_def':1914,	'hp':69542,	'specific':5+5,	'adapt':3,	'crit_dmg':3+3	}
gloves_rigo_e9r15   = {'phy_atk':1368,	'mag_atk':0,	'phy_def':1553,	'mag_def':1734,	'hp':28748,	'specific':5+5,	'adapt':3,	'crit_dmg':3+3	}
shoes_rigo_e9r15    = {'phy_atk':0,	    'mag_atk':1332,	'phy_def':1708,	'mag_def':1554,	'hp':38945,	'specific':5+5,	'adapt':3,	'crit_dmg':3+3	}

top_tene_e9r15      = {'phy_atk':1457,	'mag_atk':0,	'phy_def':2185,	'mag_def':2036,	'hp':82761,	'specific':5+5,	'adapt':0,	'crit_dmg':3+3,	'dmg_per':2}
bottom_tene_e9r15   = {'phy_atk':0,	    'mag_atk':1964,	'phy_def':2010,	'mag_def':2216,	'hp':72562,	'specific':5+5,	'adapt':0,	'crit_dmg':3+3,	'dmg_per':2}
gloves_tene_e9r15   = {'phy_atk':1670,	'mag_atk':0,	'phy_def':1835,	'mag_def':2036,	'hp':31768,	'specific':5+5,	'adapt':0,	'crit_dmg':3+3,	'dmg_per':2}
shoes_tene_e9r15    = {'phy_atk':0,	    'mag_atk':1634,	'phy_def':2010,	'mag_def':1856,	'hp':41965,	'specific':5+5,	'adapt':0,	'crit_dmg':3+3,	'dmg_per':2}


base_class_stats    = { 'phy_atk':1202,	'mag_atk':1202,	'phy_def':301,	'mag_def':301,	'hp':157000,
                        'dmg_per':      18,  
                        'dmg_per_m':    0,
                        'crit':         25,        
                        'maxi':         0,        
                        'crit_dmg':     50,

                        'pola':         0,
                        'specific':     0,
                        'all_s_dmg':    3,
                        'boss_dmg':     0,
                        'adapt':        0,
                        'cdr':          0
                       }

top = [
	Stats('top', 'Rigo top', 'Chrimson',    top_rigo_e9r15),
	Stats('top', 'Rigo top', 'Sage',        top_rigo_e9r15),
	Stats('top', 'Tene top', 'With HP',     top_tene_e9r15),
	Stats('top', 'Tene top', 'Reset',       top_tene_e9r15)
]

bottom = [
	Stats('bottom', 'Rigo bottom', 'Chrimson',  bottom_rigo_e9r15),
	Stats('bottom', 'Rigo bottom', 'Sage',      bottom_rigo_e9r15),
	Stats('bottom', 'Tene bottom', 'With HP',   bottom_tene_e9r15),
	Stats('bottom', 'Tene bottom', 'Reset',     bottom_tene_e9r15)
]

gloves = [
	Stats('gloves', 'Rigo gloves', 'Chrimson',  gloves_rigo_e9r15),
	Stats('gloves', 'Rigo gloves', 'Sage',      gloves_rigo_e9r15),
	Stats('gloves', 'Tene gloves', 'With HP',   gloves_tene_e9r15),
	Stats('gloves', 'Tene gloves', 'Reset',     gloves_tene_e9r15)
]

shoes = [
	Stats('shoes', 'Rigo shoes', 'Chrimson',    shoes_rigo_e9r15),
	Stats('shoes', 'Rigo shoes', 'Sage',        shoes_rigo_e9r15),
	Stats('shoes', 'Tene shoes', 'With HP',     shoes_tene_e9r15),
	Stats('shoes', 'Tene shoes', 'Reset',       shoes_tene_e9r15)
]
'''
ib = [
    Stats('ib_hair', 'SK Top', 'SK_armor', test_stats_2),
    Stats('ib_top', 'SK Top', 'SK_armor', test_stats_2),
    Stats('ib_bottom', 'SK Bottom', 'SK_armor', test_stats_2),
    Stats('ib_gloves', 'SK Gloves', 'SK_armor', test_stats_2),
    Stats('ib_shoes', 'SK Shoes', 'SK_armor', test_stats_2)
]
'''

available_sockets = [  
                        Stats('0', 'socket', '', {'maxi':6}),
                        Stats('0', 'socket', '', {'crit':6}),
                        Stats('0', 'socket', '', {'boss_dmg':2.5})
                    ]
sockets_combinations = list(combinations_with_replacement(available_sockets, 50))

itemized_sockets = []
with open(file_path, "w") as file:
    for i, sockets in enumerate(sockets_combinations):
        tot_socket = Stats('0', 'tot_socket', '')
        for socket in sockets:
            tot_socket += socket
            
        file.write(f'{i:10}, {tot_socket}\n')

        itemized_sockets.append(tot_socket)


combs = list(product(top, bottom, gloves, shoes, itemized_sockets))
combs_len = len(combs)

base_class_stats_item = Stats('base','','',attributes=base_class_stats)
chars = []

for i, items in enumerate(combs):
    chars.append(Character('U_%s'%i, 'jeff_%s'%i, ''))
    chars[-1] = chars[-1].equip(base_class_stats_item, base_class_stats_item.id)
    for item in items:
        chars[-1] = chars[-1].equip(item, item.id)

dataset = []
for x_jeff in chars:
	dmg, x_jeff = x_jeff.get_damage()
	dataset.append([round(dmg,0), x_jeff])

dataset.sort(key=lambda x: x[0], reverse=True)



In [2]:

best_set = chars[int(dataset[0][1].id.split('_')[-1])]
print(dataset[0][0])
print(best_set)

best_set = chars[int(dataset[1][1].id.split('_')[-1])]
print(dataset[1][0])
print(best_set)

93255.0
[id]      |name                      |set       |phy_atk   |mag_atk   |phy_def   |mag_def   |hp        |dmg_per   |dmg_per_m |crit      |maxi      |crit_dmg  |pola      |specific  |all_s_dmg |boss_dmg  |adapt     |cdr       
base      |                          |          |1202      |1202      |301       |301       |157000    |18        |0         |25        |0         |50        |0         |0         |3         |0         |0         |0         
top       |Rigo top                  |Chrimson  |1155      |0         |1883      |1734      |79741     |0         |0         |0         |0         |6         |0         |10        |0         |0         |3         |0         
bottom    |Rigo bottom               |Chrimson  |0         |1662      |1708      |1914      |69542     |0         |0         |0         |0         |6         |0         |10        |0         |0         |3         |0         
gloves    |Rigo gloves               |Chrimson  |1368      |0         |1553      |1734      

In [ ]:
with open(file_path, "w") as file:
	
	file.write(f'{"DAMAGE":<10} {Stats.headers()}\n')
	
	for dmg, stats in dataset:
		file.write(f'{dmg:<10}\n{stats}\n')